**Tutorial: Building customer churn models**

Use case examples

You can solve other binary classification problems using Amazon Redshift ML, such as predicting if a sales lead will close or not. You could also predict whether a financial transaction is fraudulent or not.

Tasks

Prerequisites
Step 1: Load the data from Amazon S3 to Amazon Redshift
Step 2: Create the machine learning model
Step 3: Perform predictions with the model

To complete this tutorial, you must have the following prerequisites:

You must set up an Amazon Redshift cluster for Amazon Redshift ML. To do so, use the documentation for Cluster and configure setup for Amazon Redshift ML administration.
The Amazon Redshift cluster that you use to create the model, and the Amazon S3 bucket that you use to stage the training data and store the model artifacts must be in the same AWS Region.


To download the SQL commands and the sample dataset used in this documentation, do one of the following:

[SQL Commands](https://s3.amazonaws.com/redshift-downloads/</target/path>/tutorial-scripts/</target/path>-tutorial.sql) ,
[Customer Activity File](https://s3.amazonaws.com/redshift-downloads/</target/path>/customer_activity/customer_activity.csv) , 
[Abalone File](https://s3.amazonaws.com/redshift-downloads/</target/path>/abalone_xg/abalone.csv)

Using the AWS CLI for Amazon S3, run the following command. You can use your own target path.

```markdown
aws s3 cp s3://redshift-downloads/</target/path>/tutorial-scripts/</target/path>-tutorial.sql </target/path>
aws s3 cp s3://redshift-downloads/</target/path>/customer_activity/customer_activity.csv </target/path>
aws s3 cp s3://redshift-downloads/</target/path>/abalone_xgb/abalone_xgb.csv </target/path>

```

**Step 1: Load the data from Amazon S3 to Amazon Redshift**

Use the Amazon Redshift query editor v2 to edit and run queries and visualize results.

Running the following queries creates a table named customer_activity and ingests the sample dataset from Amazon S3.

In [ ]:
DROP TABLE IF EXISTS customer_activity;

CREATE TABLE customer_activity (
state varchar(2),
account_length int,
area_code int,
phone varchar(8),
intl_plan varchar(3),
vMail_plan varchar(3),
vMail_message int,
day_mins float,
day_calls int,
day_charge float,
total_charge float,
eve_mins float,
eve_calls int,
eve_charge float,
night_mins float,
night_calls int,
night_charge float,
intl_mins float,
intl_calls int,
intl_charge float,
cust_serv_calls int,
churn varchar(6),
record_date date
);

COPY customer_activity
FROM 's3://</target/path>/customer_activity/'
REGION 'us-east-1' IAM_ROLE default
FORMAT AS CSV IGNOREHEADER 1;

**Step 2: Create the machine learning model**

Churn is our target input in this model. All other inputs for the model are attributes that help to create a function to predict churn.

The following example uses the CREATE MODEL operation to deliver a model that predicts whether a customer will be active, using inputs such as the customer’s age, postal code, spending, and cases. In the following example, replace DOC-EXAMPLE-BUCKET with your own Amazon S3 bucket.

In [ ]:
CREATE MODEL customer_churn_auto_model
FROM
    (
      SELECT state,
             account_length,
             area_code,
             total_charge/account_length AS average_daily_spend,
             cust_serv_calls/account_length AS average_daily_cases,
             churn
      FROM customer_activity
      WHERE  record_date < '2020-01-01'
     )
TARGET churn FUNCTION ml_fn_customer_churn_auto
IAM_ROLE default SETTINGS (
  S3_BUCKET '</target/path>ml'
);

**Check the status of model training**

You can use the SHOW MODEL command to know when your model is ready.

Use the following operation to check the status of the model.

When the model training is complete, the model_state variable becomes Model is Ready, and the prediction function becomes available.

In [ ]:
SHOW MODEL customer_churn_auto_model;

**Step 3: Perform predictions with the model**

You can use SQL statements to view the predictions made by the prediction model. In this example, the prediction function created by the CREATE MODEL operation is named ml_fn_customer_churn_auto. The input arguments for the prediction function correspond to the types of the features, such as varchar for the state and integer for account_length. The output of the prediction function is the same type as the TARGET column of the CREATE MODEL statement.

1. You trained the model on data from before 2020-01-01, so now you use the prediction function on the testing set. The following query displays the predictions of whether customers who signed up after 2020-01-01 will go through churn or not.

In [ ]:
SELECT
    phone,
    ml_fn_customer_churn_auto(
        state,
        account_length,
        area_code,
        total_charge / account_length,
        cust_serv_calls / account_length
    ) AS active
FROM
    customer_activity
WHERE
    record_date > '2020-01-01';

2. The following example uses the same prediction function for a different use case. In this case, Amazon Redshift predicts the proportion of churners and non-churners among customers from different states where the record date is greater than 2020-01-01.

In [ ]:
WITH predicted AS (
    SELECT
        state,
        ml_fn_customer_churn_auto(
            state,
            account_length,
            area_code,
            total_charge / account_length,
            cust_serv_calls / account_length
        ) :: varchar(6) AS active
    FROM
        customer_activity
    WHERE
        record_date > '2020-01-01'
)
SELECT
    state,
    SUM(
        CASE
            WHEN active = 'True.' THEN 1
            ELSE 0
        END
    ) AS churners,
    SUM(
        CASE
            WHEN active = 'False.' THEN 1
            ELSE 0
        END
    ) AS nonchurners,
    COUNT(*) AS total_per_state
FROM
    predicted
GROUP BY
    state
ORDER BY
    state;

3. The following example uses the prediction function for the use case of predicting the percentage of customers who churn in a state. In this case, Amazon Redshift predicts the churn percentage where the record date is greater than 2020-01-01.

In [ ]:
WITH predicted AS (
    SELECT
        state,
        ml_fn_customer_churn_auto(
            state,
            account_length,
            area_code,
            total_charge / account_length,
            cust_serv_calls / account_length
        ) :: varchar(6) AS active
    FROM
        customer_activity
    WHERE
        record_date > '2020-01-01'
)
SELECT
    state,
    CAST((CAST((SUM(
        CASE
            WHEN active = 'True.' THEN 1
            ELSE 0
        END
    )) AS FLOAT) / CAST(COUNT(*) AS FLOAT)) AS DECIMAL (3, 2)) AS pct_churn,
    COUNT(*) AS total_customers_per_state
FROM
    predicted
GROUP BY
    state
ORDER BY
    3 DESC;